In [2]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [3]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [4]:

TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [5]:

import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [6]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message
            
            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史
                
            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")
                
                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

In [7]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '深海蓝藻保湿面膜'}
[Tool Call] 模拟查询产品数据库：深海蓝藻保湿面膜
Observation: 工具返回结果：深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '小红书面膜推荐 2024 最新趋势 保湿面膜'}
[Tool Call] 模拟搜索网页：小红书面膜推荐 2024 最新趋势 保湿面膜
Observation: 工具返回结果：小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '深海蓝藻面膜 用户评价 保湿效果'}
[Tool Call] 模拟搜索网页：深海蓝藻面膜 用户评价 保湿效果
Observation: 工具返回结果：未找到关于 '深海蓝藻面膜 用户评价 保湿效果' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。
-- Iteration 4 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '补水保湿 惊喜效果 水光肌 护肤推荐'}
[Tool Call] 模拟生成表情符号，上下文：补水保湿 惊喜效果 水光肌 护肤推荐
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 5 --
[模型生成结果] **Observation:** 表情符号已生成，但只有4个。根据要求需要至少5个，我可以再补充一些。现在我已经收集了足够的信息来创作

In [9]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}
[Tool Call] 模拟查询产品数据库：美白精华
Observation: 工具返回结果：美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '2024美白精华趋势 小红书热门 用户评价'}
[Tool Call] 模拟搜索网页：2024美白精华趋势 小红书热门 用户评价
Observation: 工具返回结果：未找到关于 '2024美白精华趋势 小红书热门 用户评价' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '烟酰胺VC美白精华 小红书推荐 使用效果'}
[Tool Call] 模拟搜索网页：烟酰胺VC美白精华 小红书推荐 使用效果
Observation: 工具返回结果：未找到关于 '烟酰胺VC美白精华 小红书推荐 使用效果' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。
-- Iteration 4 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美白提亮 温柔知性 护肤精华 效果惊喜'}
[Tool Call] 模拟生成表情符号，上下文：美白提亮 温柔知性 护肤精华 效果惊喜
Observation: 工具返回结果：['💖', '😍', '🤩', '💯']
-- Iteration 5 --
[模型生成结果] **Observation**: 获得了4个表情符号，但用户要求至少5个。我需要补充一些适合美白护肤主题的表情符号。

**Thought**: 基于产品信息和用户要求

In [10]:

import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [11]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)

--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [12]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 救命！这面膜让我从沙漠皮秒变水光肌💦深海蓝藻真的绝了！

姐妹们！最近挖到宝了！这个深海蓝藻保湿面膜我要按头安利给所有干皮敏感肌的宝子们！

✨使用体验：
1. 质地超级清爽，一点都不粘腻，敷完脸嫩嫩滑滑的
2. 深海蓝藻提取物真的不是盖的，敷完第二天上妆完全不卡粉
3. 敏感肌友好！泛红的地方敷完立马舒缓了好多

💧核心功效：
- 深层补水：沙漠干皮救星！敷完脸像喝饱水一样
- 修护屏障：敏感肌的福音，泛红说拜拜
- 熬夜急救：第二天皮肤还是水润透亮

🌊使用小贴士：
一周2-3次，每次15分钟，敷完记得按摩吸收哦～

真的没想到这个价格能买到这么宝藏的面膜！已经囤货了，姐妹们冲鸭！

#深海蓝藻面膜 #保湿面膜推荐 #干皮救星 #敏感肌护肤 #水光肌养成 #熬夜急救面膜 #平价好物


In [ ]:
#案例1

In [13]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [14]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [15]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时潮流信息和用户反馈，用于获取最新球鞋流行趋势、穿搭风格、用户评价、市场热度等。特别针对耐克空军1号等经典鞋款，关注小红书、得物、微博等平台的真实用户声音。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的精确关键词或问题，例如'2024年耐克空军1号小红书热门趋势'、'AF1纯白款用户真实测评'、'经典板鞋搭配技巧'、'球鞋市场行情分析'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取耐克空军1号板鞋的详细产品信息，包括设计特点、核心技术、材质规格、尺码建议、保养方法等专业数据。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的精确产品名称，例如'耐克空军1号低帮纯白板鞋'、'Nike Air Force 1 '07'、'AF1小麦色限量款'，支持包含配色和款式信息"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的鞋类产品相关内容，生成一组适合小红书潮流风格的表情符号组合。针对耐克空军1号的特点，生成经典、百搭、舒适、潮流相关的表情。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'经典百搭不过时'、'Air气垫舒适脚感'、'纯白AF1易脏问题'、'OOTD潮流搭配'、'球鞋开箱惊喜'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [16]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书潮流趋势" in query or "球鞋流行趋势" in query:
        return "近期小红书潮流趋势：复古风回潮、经典款永不过时、Clean Fit穿搭风格盛行。热门关键词有#经典百搭、#OOTD、#复古球鞋、#日常穿搭。耐克空军1号作为经典鞋款，常年占据热门榜单。"
    elif "板鞋测评" in query or "板鞋推荐" in query:
        return "小红书板鞋热门话题：经典款vs联名款、舒适度对比、搭配技巧。用户痛点：磨脚、鞋底硬、容易发黄、价格波动。"
    elif "耐克空军1号" in query or "Air Force 1" in query:
        return "关于耐克空军1号的用户评价：经典百搭不过时，脚感舒适，皮质质感好。缺点：鞋底偏重，白色款容易脏，透气性一般。热门配色：纯白、黑白熊猫、小麦色。搭配建议：工装裤、牛仔裤、阔腿裤。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注鞋款设计、舒适度、搭配性和价格。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "耐克空军1号" in product_name or "Air Force 1" in product_name:
        return """耐克空军1号板鞋（Nike Air Force 1 '07）：
- 核心设计：经典低帮板鞋造型，鞋面采用优质皮革，鞋侧经典Swoosh标志，后跟Nike Air刺绣标识。
- 核心技术：内置Air Sole气垫（全掌气垫），提供缓震和舒适性；橡胶大底，耐磨防滑。
- 主要特点：经典百搭、脚感舒适、皮质质感好、鞋型硬挺不易变形。
- 适用场景：日常穿搭、休闲运动、街头潮流、校园通勤。
- 常见配色：纯白（经典中的经典）、黑白熊猫、小麦色、全黑、米白等。
- 尺码信息：标准尺码，部分用户反馈偏大半码，建议试穿或咨询客服。
- 保养建议：避免暴晒雨淋，定期清洁防发黄，使用鞋撑保持鞋型。
- 价格区间：官方售价799-899元，联名款/限定款价格更高。"""
    elif "阿迪达斯贝壳头" in product_name:
        return "阿迪达斯贝壳头板鞋：经典三叶草系列，主要特点为鞋头贝壳状设计，鞋面采用皮革材质，鞋底为橡胶材质，舒适度较好，适合日常穿搭。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "经典" in context or "百搭" in context or "复古" in context:
        return ["👟", "🔥", "💎", "✨", "🕶️"]
    elif "舒适" in context or "脚感" in context or "好穿" in context:
        return ["👍", "😌", "🦶", "🌟", "💯"]
    elif "搭配" in context or "穿搭" in context or "OOTD" in context:
        return ["👖", "👕", "🧥", "🎒", "👒"]
    elif "耐克" in context or "AF1" in context or "空军1号" in context:
        return ["✅", "🏆", "⭐", "🛍️", "💖"]
    elif "缺点" in context or "问题" in context or "吐槽" in context:
        return ["😭", "⚠️", "🤔", "💡", "🔧"]
    else:
        return random.sample(["👟", "🔥", "💎", "✨", "🕶️", "👍", "😌", "👖", "👕", "✅", "🏆", "⭐", "🛍️", "💖", "😭", "⚠️"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [17]:
# 测试案例 1: 
product_name_1 = "耐克空军1号"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：耐克空军1号，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '2024年耐克空军1号小红书热门趋势 用户真实测评'}
[Tool Call] 模拟搜索网页：2024年耐克空军1号小红书热门趋势 用户真实测评
Observation: 工具返回结果：关于耐克空军1号的用户评价：经典百搭不过时，脚感舒适，皮质质感好。缺点：鞋底偏重，白色款容易脏，透气性一般。热门配色：纯白、黑白熊猫、小麦色。搭配建议：工装裤、牛仔裤、阔腿裤。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '耐克空军1号低帮纯白板鞋'}
[Tool Call] 模拟查询产品数据库：耐克空军1号低帮纯白板鞋
Observation: 工具返回结果：耐克空军1号板鞋（Nike Air Force 1 '07）：
- 核心设计：经典低帮板鞋造型，鞋面采用优质皮革，鞋侧经典Swoosh标志，后跟Nike Air刺绣标识。
- 核心技术：内置Air Sole气垫（全掌气垫），提供缓震和舒适性；橡胶大底，耐磨防滑。
- 主要特点：经典百搭、脚感舒适、皮质质感好、鞋型硬挺不易变形。
- 适用场景：日常穿搭、休闲运动、街头潮流、校园通勤。
- 常见配色：纯白（经典中的经典）、黑白熊猫、小麦色、全黑、米白等。
- 尺码信息：标准尺码，部分用户反馈偏大半码，建议试穿或咨询客服。
- 保养建议：避免暴晒雨淋，定期清洁防发黄，使用鞋撑保持鞋型。
- 价格区间：官方售价799-899元，联名款/限定款价格更高。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': 'AF1搭配技巧 小红书OOTD 潮流穿搭 2024'}
[Tool Call] 模拟搜索网页：AF1搭配技巧 小红书OOTD 潮流穿搭 2024
Observation: 工具返回结果：未找到关于 'AF1搭配技巧 小红书OOTD 潮流穿搭 2024

In [18]:

import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [19]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 经典永不过时！耐克空军1号让我成为OOTD王者 👑",
  "body": "宝子们！今天必须按头安利我的本命鞋——耐克空军1号！💖\\n\\n作为一个潮流爱好者，我宣布AF1就是鞋柜里的定海神针！无论是工装裤、牛仔裤还是阔腿裤，它都能完美hold住，简直就是搭配神器！🤩\\n\\n🌟 核心优势：\\n✅ 经典百搭：纯白配色永远的神！一年四季都能穿，随便搭都好看\\n✅ 舒适脚感：全掌Air气垫真的绝！走一天都不累，脚感软弹舒适\\n✅ 皮质质感：优质皮革鞋面，硬挺有型，穿久了也不会变形\\n✅ 永不过时：从1982年到现在，经典地位无人能撼动！\\n\\n💡 使用体验：\\n👟 开箱瞬间就被质感惊艳到！鞋型超正，Swoosh标志超有辨识度\\n🦶 第一次上脚稍微有点硬，穿几次就完全贴合脚型了\\n👖 搭配建议：工装裤+白T是最经典的，想要时髦点可以搭阔腿裤\\n\\n⚠️ 小缺点提醒：\\n😭 纯白款真的容易脏！建议备个小白鞋清洁神器\\n😮‍💨 鞋底稍微偏重，但习惯了完全不影响走路\\n🌬️ 透气性一般，夏天穿可能会有点热\\n\\n🎯 购买建议：\\n💸 官方799-899元，建议蹲活动入手更划算\\n📏 尺码问题：部分人反馈偏大半码，建议试穿或咨询客服\\n🔧 保养tips：避免暴晒防发黄，定期清洁保持颜值\\n\\n真心推荐给所有想要经典百搭、舒适耐穿的宝子们！入股不亏！💖",
  "hashtags": ["#耐克空军1号", "#AF1", "#经典板鞋", "#OOTD", "#潮流穿搭", "#百搭神鞋", "#耐克", "#球鞋控"],
  "emojis": ["✨", "💖", "🤩", "👟", "🔥", "✅", "🦶", "👖", "💸"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)

--- 格式化后的小红书文案 (Markdown) ---
## ✨ 经典永不过时！耐克空军1号让我成为OOTD王者 👑

宝子们！今天必须按头安利我的本命鞋——耐克空军1号！💖

作为一个潮流爱好者，我宣布AF1就是鞋柜里的定海神针！无论是工装裤、牛仔裤还是阔腿裤，它都能完美hold住，简直就是搭配神器！🤩

🌟 核心优势：
✅ 经典百搭：纯白配色永远的神！一年四季都能穿，随便搭都好看
✅ 舒适脚感：全掌Air气垫真的绝！走一天都不累，脚感软弹舒适
✅ 皮质质感：优质皮革鞋面，硬挺有型，穿久了也不会变形
✅ 永不过时：从1982年到现在，经典地位无人能撼动！

💡 使用体验：
👟 开箱瞬间就被质感惊艳到！鞋型超正，Swoosh标志超有辨识度
🦶 第一次上脚稍微有点硬，穿几次就完全贴合脚型了
👖 搭配建议：工装裤+白T是最经典的，想要时髦点可以搭阔腿裤

⚠️ 小缺点提醒：
😭 纯白款真的容易脏！建议备个小白鞋清洁神器
😮‍💨 鞋底稍微偏重，但习惯了完全不影响走路
🌬️ 透气性一般，夏天穿可能会有点热

🎯 购买建议：
💸 官方799-899元，建议蹲活动入手更划算
📏 尺码问题：部分人反馈偏大半码，建议试穿或咨询客服
🔧 保养tips：避免暴晒防发黄，定期清洁保持颜值

真心推荐给所有想要经典百搭、舒适耐穿的宝子们！入股不亏！💖

#耐克空军1号 #AF1 #经典板鞋 #OOTD #潮流穿搭 #百搭神鞋 #耐克 #球鞋控

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [ ]:
#案例2

In [20]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [22]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [24]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时科技资讯和用户反馈，用于获取最新iPhone 17发布动态、性能评测、用户真实体验、市场热度、竞争对手分析等。特别关注科技媒体、数码博主、社交平台的深度评测和用户声音。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的精确关键词或问题，例如'iPhone 17最新发布信息'、'A19芯片性能评测'、'灵动岛Pro用户体验'、'铝合金机身优缺点'、'iOS 18新功能体验'、'iPhone 17 vs 华为Mate 70对比'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取iPhone 17手机的详细技术规格、硬件参数、创新功能、系统特性、价格信息等专业数据。包含官方发布信息和权威评测数据。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的精确产品名称，例如'iPhone 17 Pro Max'、'iPhone 17 标准版'、'iPhone 17 铝合金版'，支持包含存储容量和配色信息，如'iPhone 17 Pro 1TB 深空黑'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的智能手机相关内容，生成一组适合科技数码类小红书风格的表情符号组合。针对iPhone 17的特点，生成高端、科技、性能、创新、用户体验相关的表情。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'A19芯片性能怪兽'、'铝合金机身超轻手感'、'灵动岛Pro交互创新'、'4800万像素拍照惊艳'、'iOS 18智能体验'、'旗舰机皇价格争议'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [25]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书科技趋势" in query or "数码产品趋势" in query:
        return "近期小红书科技趋势：AI手机大热、铝合金材质流行、折叠屏竞争激烈、iOS 18智能生态受关注。热门关键词有#iPhone17、#A19芯片、#灵动岛Pro、#钛合金机身、#iOS18。iPhone 17作为苹果最新旗舰，占据科技话题榜首。"
    elif "旗舰手机测评" in query or "手机推荐" in query:
        return "小红书旗舰手机热门话题：性能对比、拍照效果、续航能力、系统体验。用户痛点：价格昂贵、发热问题、充电速度、重量控制。"
    elif "iPhone 17" in query or "iPhone17" in query:
        return "关于iPhone 17的用户评价：A19芯片性能怪兽，铝合金机身质感无敌，灵动岛Pro交互惊艳，拍照效果大幅提升。缺点：价格昂贵，发热问题依然存在，充电速度相对安卓较慢。热门型号：iPhone 17 Pro Max、iPhone 17 Pro、iPhone 17标准版。使用建议：搭配MagSafe配件，合理使用散热器，及时更新iOS系统。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注手机性能、拍照效果、系统体验和价格。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "iPhone 17" in product_name or "iPhone17" in product_name:
        return """iPhone 17 系列旗舰手机（Apple iPhone 17 Series）：
- 核心配置：
  • 处理器：Apple A19 Bionic芯片（3nm工艺，6核CPU+6核GPU+16核神经引擎）
  • 内存：标准版8GB，Pro版12GB，Pro Max版16GB
  • 存储：128GB/256GB/512GB/1TB/2TB（Pro系列支持最高2TB）

- 屏幕显示：
  • 标准版：6.1英寸超视网膜XDR显示屏（2556×1179，460ppi）
  • Pro版：6.1英寸ProMotion XDR显示屏（120Hz自适应刷新率）
  • Pro Max版：6.7英寸ProMotion XDR显示屏（120Hz自适应刷新率）
  • 全系支持灵动岛Pro交互功能

- 相机系统：
  • 前置：4800万像素原深感摄像头（支持面容ID）
  • 后置主摄：4800万像素（f/1.78光圈，传感器位移式光学防抖）
  • 超广角：1200万像素（f/2.2光圈，120°视角）
  • 长焦（Pro系列）：1200万像素（3倍光学变焦，15倍数码变焦）
  • 新增：AI影像引擎，支持计算摄影和实时HDR

- 机身设计：
  • 材质：铝合金中框+陶瓷护盾玻璃
  • 重量：标准版187g，Pro版206g，Pro Max版221g
  • 配色：深空黑、银色、金色、蓝色、钛合金原色
  • 防护：IP68级防尘防水（6米水深30分钟）

- 系统与功能：
  • 操作系统：iOS 18（支持AI智能助理、个性化锁屏、隐私增强）
  • 电池：标准版3279mAh，Pro版3200mAh，Pro Max版4323mAh
  • 充电：20W有线快充，15W MagSafe无线充电
  • 连接：5G、Wi-Fi 6E、蓝牙5.3、USB-C接口

- 价格区间：
  • iPhone 17：5999-8999元
  • iPhone 17 Pro：7999-11999元
  • iPhone 17 Pro Max：8999-13999元

- 使用建议：
  • 搭配MagSafe生态配件使用效果更佳
  • 建议开启优化电池充电延长寿命
  • 高强度使用建议搭配散热背夹"""
    elif "华为Mate 70" in product_name:
        return "华为Mate 70系列：搭载麒麟9100芯片，鸿蒙OS 5.0系统，XMAGE影像系统，卫星通信功能。主要特点：国产芯片突破，拍照实力强劲，商务定位明确。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "性能" in context or "芯片" in context or "速度" in context:
        return ["🚀", "🔥", "💪", "⚡", "🧠"]
    elif "设计" in context or "质感" in context or "外观" in context:
        return ["✨", "💎", "🦾", "📱", "👑"]
    elif "拍照" in context or "相机" in context or "影像" in context:
        return ["📸", "🌟", "✨", "💫", "🌈"]
    elif "系统" in context or "iOS" in context or "智能" in context:
        return ["🤖", "🧠", "💡", "🔄", "🔒"]
    elif "iPhone" in context or "苹果" in context or "旗舰" in context:
        return ["✅", "🏆", "⭐", "🛍️", "💖"]
    elif "缺点" in context or "问题" in context or "吐槽" in context:
        return ["😭", "⚠️", "🔥", "🤔", "💸"]
    elif "价格" in context or "昂贵" in context or "性价比" in context:
        return ["💸", "💰", "😬", "🤔", "📈"]
    else:
        return random.sample(["🚀", "🔥", "💎", "✨", "📱", "👍", "🤖", "📸", "✅", "🏆", "⭐", "🛍️", "💖", "😭", "⚠️", "💸"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [26]:
# 测试案例 1: 
product_name_1 = "iPhone 17"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：iPhone 17，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': 'iPhone 17最新发布信息 2024年 灵动岛Pro A19芯片 用户评测'}
[Tool Call] 模拟搜索网页：iPhone 17最新发布信息 2024年 灵动岛Pro A19芯片 用户评测
Observation: 工具返回结果：关于iPhone 17的用户评价：A19芯片性能怪兽，铝合金机身质感无敌，灵动岛Pro交互惊艳，拍照效果大幅提升。缺点：价格昂贵，发热问题依然存在，充电速度相对安卓较慢。热门型号：iPhone 17 Pro Max、iPhone 17 Pro、iPhone 17标准版。使用建议：搭配MagSafe配件，合理使用散热器，及时更新iOS系统。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': 'iPhone 17 Pro Max'}
[Tool Call] 模拟查询产品数据库：iPhone 17 Pro Max
Observation: 工具返回结果：iPhone 17 系列旗舰手机（Apple iPhone 17 Series）：
- 核心配置：
  • 处理器：Apple A19 Bionic芯片（3nm工艺，6核CPU+6核GPU+16核神经引擎）
  • 内存：标准版8GB，Pro版12GB，Pro Max版16GB
  • 存储：128GB/256GB/512GB/1TB/2TB（Pro系列支持最高2TB）

- 屏幕显示：
  • 标准版：6.1英寸超视网膜XDR显示屏（2556×1179，460ppi）
  • Pro版：6.1英寸ProMotion XDR显示屏（120Hz自适应刷新率）
  • Pro Max版：6.7英寸ProMotion XDR显示屏（120Hz自适应刷新率）
  • 全系支持灵动岛Pro交互功能

- 相机系统：
  • 前置：4800万像素原深感摄像头（支持面容ID）
  • 后置主摄：4800万像素（f/1.78

In [27]:

import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [28]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "🚀 iPhone 17 Pro Max深度体验：A19芯片性能怪兽，钛合金质感封神！",
  "body": "数码控们！终于拿到了心心念念的iPhone 17 Pro Max！作为苹果最新旗舰，这体验真的绝了！🤩\\n\\n经过一周深度使用，必须和大家分享这款性能猛兽的真实体验！从A19芯片到钛合金机身，从灵动岛Pro到4800万像素主摄，苹果这次真的把旗舰体验拉满了！💪\\n\\n🌟 核心亮点深度评测：\\n✅ A19芯片性能：3nm工艺真的太顶了！原神最高画质60帧稳如老狗，多任务切换丝滑到飞起，性能怪兽实至名归！🚀\\n✅ 钛合金机身：轻了！真的轻了！206g的重量比上代舒服太多，质感高级，手感细腻，拿在手里就是科技艺术品！💎\\n✅ 灵动岛Pro交互：支持更多第三方应用，实时信息显示更智能，交互体验比上代提升明显，真正的生产力工具！💡\\n✅ 影像系统升级：4800万像素主摄+AI影像引擎，夜景纯净度惊人，人像模式边缘处理完美，随手一拍都是大片！📸\\n✅ iOS 18智能生态：AI助理真的懂我！自动整理相册、智能提醒、个性化锁屏，苹果生态的闭环体验无人能敌！🤖\\n\\n💡 真实使用体验：\\n📱 开箱瞬间就被钛合金原色的质感惊艳到，比渲染图好看太多！\\n⚡ 日常使用完全感受不到卡顿，大型游戏加载速度提升明显\\n🔋 续航表现优秀，正常使用一天完全没问题，20W快充够用\\n📶 5G信号稳定，Wi-Fi 6E速度起飞，连接体验一流\\n\\n⚠️ 小缺点实话实说：\\n😭 价格是真的贵！Pro Max顶配接近1.4w，钱包在哭泣\\n🔥 高负载游戏还是会发热，建议搭配散热背夹使用\\n🔋 充电速度相比安卓旗舰还是慢了些，快充协议不够开放\\n💸 配件价格也贵，MagSafe生态配件又是一笔开销\\n\\n🎯 购买建议：\\n💰 预算充足直接上Pro Max，体验最完整\\n📊 存储建议：256GB起步，重度用户推荐512GB/1TB\\n🛍️ 建议官网或授权店购买，避免翻车风险\\n🔧 使用建议：开启优化电池充电，定期清理存储空间\\n\\n如果你是追求极致性能、顶级质感和完整生态体验的数码爱好者，iPhone 17绝对是你的不二之选！虽然贵，但体验真的值回票价！💖",
  "hashtags": ["#iPhone17", "#iPhone17ProMax", "#A19芯片", "#钛合金机身", "#灵动岛Pro", "#iOS18", "#苹果旗舰", "#数码评测", "#旗舰手机", "#科技好物"],
  "emojis": ["🚀", "🤩", "💪", "💎", "💡", "📸", "🤖", "📱", "⚡", "🔋", "😭", "🔥", "💸", "💖"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)

--- 格式化后的小红书文案 (Markdown) ---
## 🚀 iPhone 17 Pro Max深度体验：A19芯片性能怪兽，钛合金质感封神！

数码控们！终于拿到了心心念念的iPhone 17 Pro Max！作为苹果最新旗舰，这体验真的绝了！🤩

经过一周深度使用，必须和大家分享这款性能猛兽的真实体验！从A19芯片到钛合金机身，从灵动岛Pro到4800万像素主摄，苹果这次真的把旗舰体验拉满了！💪

🌟 核心亮点深度评测：
✅ A19芯片性能：3nm工艺真的太顶了！原神最高画质60帧稳如老狗，多任务切换丝滑到飞起，性能怪兽实至名归！🚀
✅ 钛合金机身：轻了！真的轻了！206g的重量比上代舒服太多，质感高级，手感细腻，拿在手里就是科技艺术品！💎
✅ 灵动岛Pro交互：支持更多第三方应用，实时信息显示更智能，交互体验比上代提升明显，真正的生产力工具！💡
✅ 影像系统升级：4800万像素主摄+AI影像引擎，夜景纯净度惊人，人像模式边缘处理完美，随手一拍都是大片！📸
✅ iOS 18智能生态：AI助理真的懂我！自动整理相册、智能提醒、个性化锁屏，苹果生态的闭环体验无人能敌！🤖

💡 真实使用体验：
📱 开箱瞬间就被钛合金原色的质感惊艳到，比渲染图好看太多！
⚡ 日常使用完全感受不到卡顿，大型游戏加载速度提升明显
🔋 续航表现优秀，正常使用一天完全没问题，20W快充够用
📶 5G信号稳定，Wi-Fi 6E速度起飞，连接体验一流

⚠️ 小缺点实话实说：
😭 价格是真的贵！Pro Max顶配接近1.4w，钱包在哭泣
🔥 高负载游戏还是会发热，建议搭配散热背夹使用
🔋 充电速度相比安卓旗舰还是慢了些，快充协议不够开放
💸 配件价格也贵，MagSafe生态配件又是一笔开销

🎯 购买建议：
💰 预算充足直接上Pro Max，体验最完整
📊 存储建议：256GB起步，重度用户推荐512GB/1TB
🛍️ 建议官网或授权店购买，避免翻车风险
🔧 使用建议：开启优化电池充电，定期清理存储空间

如果你是追求极致性能、顶级质感和完整生态体验的数码爱好者，iPhone 17绝对是你的不二之选！虽然贵，但体验真的值回票价！💖

#iPhone17 #iPhone17ProMax #A19芯片 #钛合金机身 #灵动岛Pro #iOS18 #苹果旗舰 #数码评测 #旗舰手机 #科技好物


In [ ]:
#案例3

In [29]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [ ]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

In [30]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时汽车资讯和用户反馈，用于获取最新奔驰S级车型信息、市场评价、车主真实体验、驾驶感受、豪华配置、竞争对手对比等。特别关注汽车媒体、车主论坛、社交平台的深度评测和用户声音。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的精确关键词或问题，例如'2024款奔驰S级最新评测'、'S500L驾驶体验'、'MBUX智能座舱使用感受'、'魔术车身控制系统优缺点'、'奔驰S级 vs 宝马7系对比'、'豪华轿车市场行情'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取奔驰S级轿车的详细车型信息，包括动力配置、豪华科技、安全系统、舒适配置、车身尺寸、价格区间等专业数据。包含官方配置信息和权威汽车评测数据。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的精确车型名称，例如'奔驰S 500 L 4MATIC'、'奔驰S 450 L'、'奔驰S 580 e 插电混动'，支持包含年款和配置信息，如'2024款奔驰S 500 L 豪华型'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的豪华汽车相关内容，生成一组适合汽车测评类小红书风格的表情符号组合。针对奔驰S级的特点，生成豪华、科技、舒适、性能、商务相关的表情。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'旗舰级豪华舒适'、'MBUX智能座舱科技感'、'3.0T直六发动机性能'、'魔术车身极致平稳'、'柏林之声音响震撼'、'百万级豪华轿车价格'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [32]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书汽车趋势" in query or "豪华轿车趋势" in query:
        return "近期小红书汽车趋势：智能座舱大热、豪华舒适体验受关注、新能源汽车竞争激烈、传统豪华品牌转型。热门关键词有#奔驰S级、#豪华轿车、#MBUX智能座舱、#魔术车身、#柏林之声。奔驰S级作为豪华轿车标杆，常年占据高端话题榜首。"
    elif "豪华轿车测评" in query or "豪华车推荐" in query:
        return "小红书豪华轿车热门话题：舒适性对比、智能科技体验、驾驶感受、品牌溢价。用户痛点：价格昂贵、油耗偏高、维修保养成本高、科技配置学习成本。"
    elif "奔驰S级" in query or "Mercedes-Benz S-Class" in query:
        return "关于奔驰S级的用户评价：旗舰级豪华舒适，MBUX智能座舱科技感强，魔术车身平稳性无敌，柏林之声音响效果震撼。缺点：价格昂贵，油耗偏高，维修保养费用高，车机系统偶尔卡顿。热门车型：S 500 L 4MATIC、S 450 L、S 580 e插电混动。使用建议：注重舒适性选空气悬架舒适模式，商务接待后排体验最佳，定期保养维护车辆状态。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注汽车舒适性、科技配置、驾驶体验和价格。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "奔驰S级" in product_name or "S-Class" in product_name:
        return """奔驰S级轿车（Mercedes-Benz S-Class）：
- 车型定位：旗舰级豪华行政轿车，D级豪华轿车市场标杆

- 动力系统：
  • S 450 L：3.0T直列六缸发动机+48V轻混系统，最大功率270kW（367马力），峰值扭矩500N·m
  • S 500 L 4MATIC：3.0T直列六缸发动机+48V轻混系统，最大功率320kW（435马力），峰值扭矩520N·m
  • S 580 e 4MATIC：3.0T直列六缸发动机+插电混动系统，综合功率375kW（510马力），纯电续航100km
  • 全系匹配9G-TRONIC自动变速箱，部分车型配备4MATIC全时四驱系统

- 车身尺寸：
  • 长×宽×高：5320×1921×1503mm（长轴距版）
  • 轴距：3216mm（长轴距版）
  • 整备质量：2100-2300kg（根据配置不同）

- 豪华科技配置：
  • MBUX智能人机交互系统：12.8英寸OLED中控屏+12.3英寸全液晶仪表
  • 增强现实抬头显示（AR-HUD）：导航信息投影至前挡风玻璃
  • 智能数字大灯：260万像素分辨率，智能光束控制
  • 魔术车身控制系统（Magic Body Control）：主动式液压悬架，可扫描路面并调整悬架
  • 柏林之声4D环绕音响系统：31个扬声器，总功率1750W
  • 多区智能空调：带香氛系统和负离子发生器

- 舒适性配置：
  • Nappa真皮座椅：前排座椅带通风/加热/按摩功能
  • 行政级后排：电动调节座椅、后排娱乐系统、无线充电
  • 全景天窗、电动吸合门、无钥匙进入、电动后备厢

- 安全配置：
  • PRE-SAFE预防性安全系统
  • 主动刹车辅助、车道保持辅助、盲点监测
  • 夜视辅助系统、交通标志识别
  • 360度全景影像、自动泊车

- 价格区间：
  • S 450 L：约110-130万元
  • S 500 L 4MATIC：约140-160万元
  • S 580 e 4MATIC：约150-170万元
  • 顶配车型价格可达200万元以上

- 使用建议：
  • 商务接待：后排舒适性最佳，注重柏林之声音响和座椅舒适性
  • 日常驾驶：建议开启舒适模式，享受魔术车身平稳性
  • 保养维护：建议在授权4S店定期保养，注意空气悬架维护"""
    elif "宝马7系" in product_name:
        return "宝马7系：宝马旗舰豪华轿车，搭载iDrive 8.0系统，后排31英寸悬浮屏幕，水晶内饰组件。主要特点：驾驶乐趣突出，科技配置丰富，后排舒适性提升。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "豪华" in context or "舒适" in context or "旗舰" in context:
        return ["🏆", "💎", "✨", "🛋️", "👑"]
    elif "科技" in context or "智能" in context or "MBUX" in context:
        return ["🚀", "💡", "🤖", "🖥️", "🔧"]
    elif "性能" in context or "动力" in context or "驾驶" in context:
        return ["🔥", "⚡", "🏎️", "💪", "🛣️"]
    elif "安全" in context or "配置" in context or "系统" in context:
        return ["🛡️", "✅", "🔒", "🚨", "📊"]
    elif "奔驰" in context or "S级" in context or "豪华轿车" in context:
        return ["✅", "🏆", "⭐", "🛍️", "💖"]
    elif "缺点" in context or "问题"in context or "吐槽" in context:
        return ["😭", "⚠️", "⛽", "🔧", "💸"]
    elif "价格" in context or "昂贵" in context or "性价比" in context:
        return ["💸", "💰", "😬", "🤔", "📈"]
    elif "音响" in context or "柏林之声" in context:
        return ["🎵", "🎶", "🎧", "🔊", "✨"]
    elif "车身" in context or "平稳" in context or "悬架" in context:
        return ["🌟", "🛡️", "⚖️", "🚗", "😌"]
    else:
        return random.sample(["🏆", "💎", "✨", "🚀", "🔥", "⚡", "✅", "⭐", "💖", "😭", "⚠️", "💸", "🎵", "🎶", "🚗", "🛋️"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

In [33]:
# 测试案例 1: 
product_name_1 = "奔驰S级"
tone_style_1 = "知性"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：奔驰S级，风格：知性

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': '2024款奔驰S级最新评测 豪华配置 驾驶体验 车主真实感受'}
[Tool Call] 模拟搜索网页：2024款奔驰S级最新评测 豪华配置 驾驶体验 车主真实感受
Observation: 工具返回结果：关于奔驰S级的用户评价：旗舰级豪华舒适，MBUX智能座舱科技感强，魔术车身平稳性无敌，柏林之声音响效果震撼。缺点：价格昂贵，油耗偏高，维修保养费用高，车机系统偶尔卡顿。热门车型：S 500 L 4MATIC、S 450 L、S 580 e插电混动。使用建议：注重舒适性选空气悬架舒适模式，商务接待后排体验最佳，定期保养维护车辆状态。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '奔驰S 500 L 4MATIC'}
[Tool Call] 模拟查询产品数据库：奔驰S 500 L 4MATIC
Observation: 工具返回结果：产品数据库中未找到关于 '奔驰S 500 L 4MATIC' 的详细信息。
-- Iteration 3 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '奔驰S级'}
[Tool Call] 模拟查询产品数据库：奔驰S级
Observation: 工具返回结果：奔驰S级轿车（Mercedes-Benz S-Class）：
- 车型定位：旗舰级豪华行政轿车，D级豪华轿车市场标杆

- 动力系统：
  • S 450 L：3.0T直列六缸发动机+48V轻混系统，最大功率270kW（367马力），峰值扭矩500N·m
  • S 500 L 4MATIC：3.0T直列六缸发动机+48V轻混系统，最大功率320kW（435马力），峰值扭矩520N·m
  • S 580 e 4MATIC：3.0T直列六缸发动机+插电混动系统，综合功率375kW（510马力）

In [35]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [36]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "🏆 奔驰S 500 L深度体验：MBUX智能座舱封神，魔术车身舒适度天花板！",
  "body": "车友们！终于深度体验了奔驰S 500 L 4MATIC！作为豪华轿车的标杆，这体验真的颠覆认知！🤩\\n\\n经过一周城市通勤+高速长途实测，必须和大家分享这款行政级座驾的真实感受！从MBUX智能座舱到魔术车身控制，从柏林之声音响到3.0T直六动力，奔驰把豪华体验做到了极致！💎\\n\\n🌟 核心亮点深度评测：\\n✅ MBUX智能座舱：12.8英寸OLED巨屏+12.3英寸全液晶仪表，语音控制精准度超高，AR-HUD导航信息直接投影前挡风玻璃，科技感拉满！🚀\\n✅ 魔术车身控制系统：主动液压悬架真的神！能扫描路面提前调整阻尼，过减速带如履平地，高速过弯支撑性完美，舒适和运动完美平衡！✨\\n✅ 柏林之声4D音响：31个扬声器+座椅振动单元，现场感震撼到哭！低音下潜深，人声清澈，简直就是移动音乐厅！🎵\\n✅ 3.0T直六+48V轻混动力：320kW最大功率，起步轻盈，中后段加速底气十足，9AT变速箱换挡平顺，油耗表现优秀！⚡\\n✅ 行政级后排舒适：Nappa真皮座椅带通风加热按摩，后排娱乐系统+无线充电，老板键一键尊享，长途乘坐零疲劳！🛋️\\n\\n💡 真实驾驶体验：\\n🚗 开箱就被气场震撼！修长车身+满天星中网，停在酒店门口回头率爆表\\n🛣️ 城市通勤舒适性无敌，魔术车身过滤90%颠簸，隔音效果顶级\\n🏎️ 高速巡航稳定性一流，4MATIC四驱系统雨天雪天信心十足\\n🔋 油耗表现：综合9.5L/100km，48V轻混系统对油耗优化明显\\n\\n⚠️ 小缺点实话实说：\\n😭 价格是真的贵！S 500 L落地接近160w，钱包在滴血\\n⛽ 市区拥堵油耗偏高，长期使用成本不容忽视\\n🔧 维修保养费用昂贵，空气悬架等部件维修成本高\\n🤔 车机系统偶尔卡顿，MBUX复杂功能学习成本较高\\n\\n🎯 购买建议：\\n💰 预算充足直接上S 500 L 4MATIC，体验最完整\\n🏢 商务接待首选，后排舒适性让客户赞不绝口\\n🚙 注重驾驶体验可选运动模式，兼顾舒适和操控\\n🔧 建议在授权4S店保养，空气悬架等部件需专业维护\\n\\n如果你是追求极致舒适、顶级科技和行政级尊贵体验的车主，奔驰S级绝对是你的不二之选！虽然贵，但体验真的对得起价格！💖",
  "hashtags": ["#奔驰S级", "#奔驰S500L", "#MBUX智能座舱", "#魔术车身", "#柏林之声", "#豪华轿车", "#行政级座驾", "#汽车评测", "#奔驰", "#豪车体验"],
  "emojis": ["🏆", "🤩", "💎", "🚀", "✨", "🎵", "⚡", "🛋️", "🚗", "🛣️", "😭", "⛽", "🔧", "💖"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)

--- 格式化后的小红书文案 (Markdown) ---
## 🏆 奔驰S 500 L深度体验：MBUX智能座舱封神，魔术车身舒适度天花板！

车友们！终于深度体验了奔驰S 500 L 4MATIC！作为豪华轿车的标杆，这体验真的颠覆认知！🤩

经过一周城市通勤+高速长途实测，必须和大家分享这款行政级座驾的真实感受！从MBUX智能座舱到魔术车身控制，从柏林之声音响到3.0T直六动力，奔驰把豪华体验做到了极致！💎

🌟 核心亮点深度评测：
✅ MBUX智能座舱：12.8英寸OLED巨屏+12.3英寸全液晶仪表，语音控制精准度超高，AR-HUD导航信息直接投影前挡风玻璃，科技感拉满！🚀
✅ 魔术车身控制系统：主动液压悬架真的神！能扫描路面提前调整阻尼，过减速带如履平地，高速过弯支撑性完美，舒适和运动完美平衡！✨
✅ 柏林之声4D音响：31个扬声器+座椅振动单元，现场感震撼到哭！低音下潜深，人声清澈，简直就是移动音乐厅！🎵
✅ 3.0T直六+48V轻混动力：320kW最大功率，起步轻盈，中后段加速底气十足，9AT变速箱换挡平顺，油耗表现优秀！⚡
✅ 行政级后排舒适：Nappa真皮座椅带通风加热按摩，后排娱乐系统+无线充电，老板键一键尊享，长途乘坐零疲劳！🛋️

💡 真实驾驶体验：
🚗 开箱就被气场震撼！修长车身+满天星中网，停在酒店门口回头率爆表
🛣️ 城市通勤舒适性无敌，魔术车身过滤90%颠簸，隔音效果顶级
🏎️ 高速巡航稳定性一流，4MATIC四驱系统雨天雪天信心十足
🔋 油耗表现：综合9.5L/100km，48V轻混系统对油耗优化明显

⚠️ 小缺点实话实说：
😭 价格是真的贵！S 500 L落地接近160w，钱包在滴血
⛽ 市区拥堵油耗偏高，长期使用成本不容忽视
🔧 维修保养费用昂贵，空气悬架等部件维修成本高
🤔 车机系统偶尔卡顿，MBUX复杂功能学习成本较高

🎯 购买建议：
💰 预算充足直接上S 500 L 4MATIC，体验最完整
🏢 商务接待首选，后排舒适性让客户赞不绝口
🚙 注重驾驶体验可选运动模式，兼顾舒适和操控
🔧 建议在授权4S店保养，空气悬架等部件需专业维护

如果你是追求极致舒适、顶级科技和行政级尊贵体验的车主，奔驰S级绝对是你的不二之选！虽然贵，但体验真的对得起价格！💖

#奔驰S级 #奔驰S500L #MBUX智能座舱 #魔术车身 #柏林